Nom: David Morillo Massagué
NIU: 1666540

# Disseny d'Aplicacions Massives de Dades - Redis lab

In [43]:
!apt install -y redis
!pip install redis
!redis-server --daemonize yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
redis is already the newest version (5:6.0.16-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [44]:
import redis
c=redis.Redis(host='localhost', port=6379)
c.ping()

True

# STRINGS
**Exercise 1**: add three of your favorite websites to the list with SET and with MSET.

In [45]:
c.set('Jesucristo', 'https://open.spotify.com/intl-es/track/4Qn4NJZZhMQVxVROSb0Je1?si=9ec96e1f51114be4')

c.mset({
    "FCB": "https://www.fcbarcelona.cat/ca/",
    "Nano": "https://www.formula1.com/en/drivers/fernando-alonso",
    "UFC": "https://www.ufc.com/events#events-list-upcoming"
})

print(c.get("FCB").decode())

https://www.fcbarcelona.cat/ca/


# HASHES
**Exercise 2**: add yourself to a new hash object using your username and a new password

In [46]:
user = "user:007"
c.hset(user, "name", "David")
c.hset(user, "age", 20)
c.hset(user, "password", "hello1234")

0

# SETS
**Exercise 3**: add a new type of URL called video and add there some examples: youtube.com, vimeo.com, netflix.com. Create a new dataset by joining tech and video websites from the previous example.

In [47]:
c.sadd("tech","theverge.com")
c.sadd("tech","cnet.com")
tech=c.smembers("tech")

c.sadd("video", "youtube.com")
c.sadd("video", "vimeo.com")
c.sadd("video", "netflix.com")
video=c.smembers("video")

c.sunionstore("tech-video", "tech", "video")
join=c.smembers("tech-video")
print(join)

{b'cnet.com', b'vimeo.com', b'theverge.com', b'netflix.com', b'youtube.com'}


# SORTED SETS
**Exercise 4**: add two new short URLs to popular sorted list with their initial ranking values

In [48]:
score_zset='votes:'
c.zadd(score_zset, {'uab' : 5})
c.zadd(score_zset, {'google': 1000})
c.zadd(score_zset, {'bing': 10})

c.zadd(score_zset, {'youtube' : 777})
c.zadd(score_zset, {'twitch': 3141})

0

**Exercise 5**: add a new item to popular ZSET with the value 9999. Check that the new item is inserted in popular by obtaining the top 5 list of items with largest decreasing score.


In [49]:
c.zadd(score_zset, {'tiktok': 9999})

top5 = c.zrevrange(score_zset, 0, 4, withscores=True)
print(top5)

[(b'tiktok', 9999.0), (b'twitch', 3141.0), (b'google', 1000.0), (b'youtube', 777.0), (b'bing', 10.0)]


# ZUNION
**Exercise 6**: apply the following union operation

In [50]:
visit_zset='visits:'
c.zadd(visit_zset, {'uab' : 20})
c.zadd(visit_zset, {'google' : 1234})

c.zunionstore('relevance:', ['votes:','visits:'])
c.zrange('relevance:', 0, 9, withscores=True)

[(b'bing', 10.0),
 (b'uab', 25.0),
 (b'youtube', 777.0),
 (b'google', 2234.0),
 (b'twitch', 3141.0),
 (b'tiktok', 9999.0)]

# LISTS

In [51]:
wishlist_pepe="pepe:wishlist"
c.lpush(wishlist_pepe, 'google')
c.lpush(wishlist_pepe, 'bing')

webs = c.lrange(wishlist_pepe, 0, -1)
print(webs)

num_len=c.llen(wishlist_pepe)
print(num_len)

[b'bing', b'google', b'bing', b'google', b'bing', b'google']
6


# Questions due to April 27th 2025:

# 2. Top 10 usuaris mes actius:

- Definició d’activitat:
Es defineix com publicar una nova URL o votar per una URL (només un cop per URL). Cada acció vàlida suma un punt d’activitat.

- Estructures Redis utilitzades:
ZSET: user_activity → emmagatzema l'activitat de cada usuari (la puntuació és el nombre d'accions).

SET: url_votes:<url> → llista d’usuaris que han votat una URL per evitar recompte duplicat.

- Operacions:
ZINCRBY user_activity 1 <user> → incrementa l'activitat

SADD url_votes:<url> <user> → afegeix l’usuari al conjunt de votants d’una URL

ZREVRANGE user_activity 0 9 WITHSCORES → retorna el top 10

- Reacció davant una nova acció:
Quan l’usuari fa una acció (post o vot), s’actualitza el ZSET user_activity en temps real. Això manté sempre actualitzat el top 10.

- Prevenció de vots duplicats:
Si l’usuari ja està al SET url_votes:<url>, no es torna a incrementar la seva activitat. Això es controla amb SISMEMBER.

In [52]:
client = redis.Redis()

# Funció per publicar una URL
def publica(user, url):
    # Incrementa activitat de l'usuari
    client.zincrby("user_activity", 1, user)

# Funció per votar una URL (només una vegada)
def vota(user, url):
    if not client.sismember(f"url_votes:{url}", user):
        client.sadd(f"url_votes:{url}", user)
        client.zincrby("user_activity", 1, user)

# Obtenir top 10 usuaris més actius
def top_usuaris():
    return client.zrevrange("user_activity", 0, 9, withscores=True)

# Exemple d'ús
publica("user123", "uab.cat")
vota("user123", "google.com")
vota("user123", "google.com")  # No compta dues vegades
vota("user456", "google.com")

print(top_usuaris())

[(b'user123', 6.0), (b'user456', 1.0)]


# 3. Ultims 10 llocs visitats per usuari

- Objectiu:
Guardar les últimes 10 webs visitades per cada usuari, ordenades per temps (la més recent primer).

- Estructures Redis:
LIST: recent:<usuari> → conté les URLs visitades en ordre cronològic invers.

- Operacions:
LREM recent:<user> 0 <url> → elimina la URL si ja existia per evitar duplicats

LPUSH recent:<user> <url> → afegeix la URL al principi de la llista

LTRIM recent:<user> 0 9 → manté només les 10 últimes visites

LRANGE recent:<user> 0 -1 → retorna les visites en ordre

- Reacció davant nova visita:
Quan hi ha un nou esdeveniment (user123, "www.google.com"), la URL es col·loca al principi de la llista. Si ja hi era, es reordena per mostrar que ha estat la darrera visitada.

In [53]:
# Funció per registrar una visita
def visita(user, url):
    key = f"recent:{user}"
    # Si ja existia, l'eliminem per evitar duplicats
    client.lrem(key, 0, url)
    # Afegim al principi
    client.lpush(key, url)
    # Mantenim només les 10 més recents
    client.ltrim(key, 0, 9)

# Llista de les últimes 10 webs visitades
def ultimes_visites(user):
    return client.lrange(f"recent:{user}", 0, -1)

# Exemple d'ús
visita("user123", "google.com")
visita("user123", "youtube.com")
visita("user123", "google.com")  # Es mou al principi

print([x.decode() for x in ultimes_visites("user123")])

['google.com', 'youtube.com']


# 4. Sistema de recomanacions

- Objectiu:
Recomanar URLs a un usuari basant-se en preferències compartides amb altres usuaris.

- Estructures Redis:
SET: likes:<usuari> → llista de URLs que l’usuari ha votat o marcat com a preferides

- Operacions:
SADD likes:<usuari> <url> → afegir una nova URL com a preferida

SINTER likes:<usuari> likes:<altre> → troba interessos comuns entre usuaris

SDIFF likes:<altre> likes:<usuari> → recomana URLs que el segon ha votat però el primer no

- Buscar usuaris similars:
Es recorre la base de dades (SCAN_ITER) per comparar conjunts likes:<user> i detectar coincidències.

- Com fer recomanacions:
Un cop detectats usuaris amb interessos compartits, es recomanen les seves URLs exclusives que l’usuari actual encara no ha visitat.

In [54]:
# Afegir URL favorita a un usuari
def afegeix_like(user, url):
    client.sadd(f"likes:{user}", url)

# Recomanar URLs noves basades en usuaris similars
def recomanacions(user):
    usuaris = [u.decode().split(":")[1] for u in client.scan_iter("likes:*")]
    likes_usuari = f"likes:{user}"
    recomanades = set()

    for altre in usuaris:
        if altre == user:
            continue
        # Si tenen interessos en comú
        if client.sinter(likes_usuari, f"likes:{altre}"):
            # Afegim les URLs de l’altre usuari que aquest no ha vist
            noves = client.sdiff(f"likes:{altre}", likes_usuari)
            recomanades.update(noves)

    return [x.decode() for x in recomanades]

# Exemple d'ús
afegeix_like("user123", "youtube.com")
afegeix_like("user123", "google.com")

afegeix_like("user456", "youtube.com")
afegeix_like("user456", "reddit.com")
afegeix_like("user456", "medium.com")

print(recomanacions("user123"))

['medium.com', 'reddit.com']
